# RoadSight — Dataset Exploration

This notebook is used to:
- Understand the RDD2020 dataset
- Inspect images and labels
- Learn how ML data is structured

No training happens here.

In [1]:
# We import standard Python libraries
# These help us interact with the operating system

import os
import sys

# Print Python version to ensure reproducibility
print("Python version:", sys.version)

# Print current working directory
# This helps us confirm where the notebook is running
print("Current directory:", os.getcwd())


Python version: 3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]
Current directory: c:\Users\saima\OneDrive\Desktop\RoadSight\RoadSight\ml\notebooks
